# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [28]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-f5d98019db-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Tennis explorative search") 

Consider the following exploratory information need:

> investigate tennis and find the main BGPs related to this sport. Compare players per countries, awards of the players and tournament.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q1426`| Roger Federer       | node |
| `wd:Q11459`| Serena Williams       | node |





Also consider

```
wd:Q1426 ?p ?obj .
```

is the BGP to retrieve all **properties of Roger Federer**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for tennis player

2. Identify the BGP for tennis tournament

3. Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

4. Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

5. Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners (hint: men and women do not play together, there are two distinct tournament ) (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

6. Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

7. Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

8. Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

## Task 1
Identify the BGP for tennis player

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1426 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1284'), ('name', 'Munzinger person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1352'), ('name', 

In [31]:

queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
  
   wd:Q1426 ?p ?obj .
   
   ?p sc:name ?name.
    ?obj sc:name ?objName.
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('name', 'victory'), ('obj', 'http://www.wikidata.org/entity/Q22956224'), ('objName', "2017 Australian Open – Men's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('name', 'victory'), ('obj', 'http://www.wikidata.org/entity/Q612597'), ('objName', '2007 Tennis Masters Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('name', 'significant person'), ('obj', 'http://www.wikidata.org/entity/Q54584'), ('objName', 'Andy Roddick')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received'), ('obj', 'http://www.wikidata.org/entity/Q56509472'), ('objName', 'honorary doctor of the University of Basel')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q589310'), ('objName', "tennis at the 2000 Summer Olympics – men's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P6758'), ('name', 'supported sports team'), ('ob

In [34]:
#Final query for this task
# P106 'occupation' 
queryString = """
SELECT DISTINCT  ?obj ?objName
WHERE {
  
   wd:Q1426 wdt:P106 ?obj .
  
   ?obj sc:name ?objName.
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q10833314'), ('objName', 'tennis player')]
1


## Task 2
Identify the BGP for tennis tournament

In [ ]:
# write your queries

In [3]:
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   
   wd:Q1426 ?p ?obj .
   
   ?p sc:name ?name.
   ?obj sc:name ?objName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('name', 'victory'), ('obj', 'http://www.wikidata.org/entity/Q22956224'), ('objName', "2017 Australian Open – Men's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('name', 'victory'), ('obj', 'http://www.wikidata.org/entity/Q612597'), ('objName', '2007 Tennis Masters Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('name', 'significant person'), ('obj', 'http://www.wikidata.org/entity/Q54584'), ('objName', 'Andy Roddick')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received'), ('obj', 'http://www.wikidata.org/entity/Q56509472'), ('objName', 'honorary doctor of the University of Basel')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q589310'), ('objName', "tennis at the 2000 Summer Olympics – men's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P6758'), ('name', 'supported sports team'), ('ob

In [6]:
# from P2522 = 'victory' Q22956224 = "2017 Australian Open – Men's Singles"

queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   
   wd:Q22956224 ?p ?obj.
   
   ?p sc:name ?name.
   ?obj sc:name ?objName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q22077628'), ('objName', '2017 Australian Open')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('obj', 'http://www.wikidata.org/entity/Q408'), ('objName', 'Australia')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q1426'), ('objName', 'Roger Federer')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class'), ('obj', 'http://www.wikidata.org/entity/Q16893072'), ('objName', "men's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q46190676'), ('objName', 'tennis event')]
6


In [7]:
# Q46190676 'tennis event'
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   
   wd:Q46190676 ?p ?obj.
   
   ?p sc:name ?name.
   ?obj sc:name ?objName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class'), ('obj', 'http://www.wikidata.org/entity/Q21170330'), ('objName', 'tennis qualification event')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P361'), ('objName', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q16510064'), ('objName', 'sporting event')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P641'), ('objName', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q47345468'), ('objName', 'tennis tournament edition')]
[('p', 'http

Final query for this task

In [10]:
# Q47345468 'tennis tournament'
queryString = """
SELECT DISTINCT ?tournament ?tournamentName
WHERE {
   
    ?tournament ?p wd:Q47345468.
   
   ?p sc:name ?name.
   ?tournament sc:name ?tournamentName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('tournament', 'http://www.wikidata.org/entity/Q1177995'), ('tournamentName', '1982 Davis Cup')]
[('tournament', 'http://www.wikidata.org/entity/Q1178014'), ('tournamentName', '1985 Davis Cup')]
[('tournament', 'http://www.wikidata.org/entity/Q1364742'), ('tournamentName', '2007 Tennis Channel Open')]
[('tournament', 'http://www.wikidata.org/entity/Q1507711'), ('tournamentName', '2010 Heineken Open')]
[('tournament', 'http://www.wikidata.org/entity/Q1508586'), ('tournamentName', '2011 Regions Morgan Keegan Championships and the Cellular South Cup')]
[('tournament', 'http://www.wikidata.org/entity/Q1520782'), ('tournamentName', '2000 Majorca Open')]
[('tournament', 'http://www.wikidata.org/entity/Q1596076'), ('tournamentName', '2012 Heineken Open')]
[('tournament', 'http://www.wikidata.org/entity/Q1644643'), ('tournamentName', '2006 Brasil Open')]
[('tournament', 'http://www.wikidata.org/entity/Q1761880'), ('tournamentName', '2007 Open de Tenis Comunidad Valenciana')]
[('tourna

## Task 3
Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

In [18]:
# Q268871  '2004 Wimbledon Championships' 
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE { 
    wd:Q268871 ?p ?obj.
   
   ?p sc:name ?name.
   optional {?obj sc:name ?objName.}
    
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q8188255'), ('objName', 'Category:2004 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('obj', 'http://www.wikidata.org/entity/Q145'), ('objName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q47345468'), ('objName', 'tennis tournament edition')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by'), ('obj', 'http://www.wikidata.org/entity/Q746693'), ('objName', '2005 Wimbledon Championships

Final query for this task

In [21]:
# Q268871  '2004 Wimbledon Championships' 
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE { 
   ?p ?obj wd:Q268871.
   
   ?p sc:name ?name.
   optional {?obj sc:name ?objName.}
    
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q746693'), ('name', '2005 Wimbledon Championships'), ('obj', 'http://www.wikidata.org/prop/direct/P155'), ('objName', 'follows')]
[('p', 'http://www.wikidata.org/entity/Q832484'), ('name', '2003 Wimbledon Championships'), ('obj', 'http://www.wikidata.org/prop/direct/P156'), ('objName', 'followed by')]
[('p', 'http://www.wikidata.org/entity/Q8188255'), ('name', 'Category:2004 Wimbledon Championships'), ('obj', 'http://www.wikidata.org/prop/direct/P301'), ('objName', "category's main topic")]
[('p', 'http://www.wikidata.org/entity/Q388810'), ('name', '2004 Wimbledon Championships - Mixed Doubles'), ('obj', 'http://www.wikidata.org/prop/direct/P361'), ('objName', 'part of')]
[('p', 'http://www.wikidata.org/entity/Q3994898'), ('name', "2004 Wimbledon Championships – Men's Doubles"), ('obj', 'http://www.wikidata.org/prop/direct/P361'), ('objName', 'part of')]
[('p', 'http://www.wikidata.org/entity/Q20656705'), ('name', "2004 Wimbledon Champions

In [27]:
#Now try 'Q41520 Wimbledon Championships' 

queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE { 
   ?p ?obj wd:Q41520.
   
   ?p sc:name ?name.
   optional {?obj sc:name ?objName.}
   filter regex(?name, "Grand Slam")
    
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q102113'), ('name', 'Grand Slam of tennis'), ('obj', 'http://www.wikidata.org/prop/direct/P527'), ('objName', 'has part')]
1


In [44]:
#Q102113  'Grand Slam of tennis' P527  'has part' 
queryString = """
SELECT DISTINCT ?GrandSlam ?GrandSlamName
WHERE { 
   wd:Q102113 wdt:P527 ?GrandSlam.
   
   ?GrandSlam sc:name ?GrandSlamName.
    
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('GrandSlam', 'http://www.wikidata.org/entity/Q2223649'), ('GrandSlamName', 'French cheese')]
[('GrandSlam', 'http://www.wikidata.org/entity/Q123577'), ('GrandSlamName', 'US Open')]
[('GrandSlam', 'http://www.wikidata.org/entity/Q41520'), ('GrandSlamName', 'Wimbledon Championships')]
[('GrandSlam', 'http://www.wikidata.org/entity/Q60874'), ('GrandSlamName', 'Australian Open')]
4


the Q2223649  'French cheese' is WRONG, so try in other way

In [45]:
#Q102113  'Grand Slam of tennis' P527  'has part' 
queryString = """
SELECT DISTINCT ?GrandSlam ?GrandSlamName ?p ?name
WHERE { 
    ?GrandSlam ?p wd:Q102113.
   
   ?GrandSlam sc:name ?GrandSlamName.
   ?p sc:name ?name.
    
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('GrandSlam', 'http://www.wikidata.org/entity/Q5851433'), ('GrandSlamName', 'Template:Infobox GrandSlamTournaments'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('GrandSlam', 'http://www.wikidata.org/entity/Q105868'), ('GrandSlamName', 'Gabriela Sabatini'), ('p', 'http://www.wikidata.org/prop/direct/P2522'), ('name', 'victory')]
[('GrandSlam', 'http://www.wikidata.org/entity/Q105838'), ('GrandSlamName', 'Marion Bartoli'), ('p', 'http://www.wikidata.org/prop/direct/P2522'), ('name', 'victory')]
[('GrandSlam', 'http://www.wikidata.org/entity/Q11300305'), ('GrandSlamName', 'Grand Slam'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('GrandSlam', 'http://www.wikidata.org/entity/Q18156144'), ('GrandSlamName', 'Category:Grand Slam (tennis)'), ('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('GrandSlam', 'http://www.wikidata.org/entity/Q41520'), ('GrandSlamName', 'Wimbledo

In [46]:
# final Query
# P361 part of'
#Final query for this task
#Q102113  'Grand Slam of tennis' P527  'has part' 
queryString = """
SELECT DISTINCT ?GrandSlam ?GrandSlamName
WHERE { 
    ?GrandSlam wdt:P361 wd:Q102113.
   
   ?GrandSlam sc:name ?GrandSlamName.
   
    
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('GrandSlam', 'http://www.wikidata.org/entity/Q43605'), ('GrandSlamName', 'French Open')]
[('GrandSlam', 'http://www.wikidata.org/entity/Q123577'), ('GrandSlamName', 'US Open')]
[('GrandSlam', 'http://www.wikidata.org/entity/Q41520'), ('GrandSlamName', 'Wimbledon Championships')]
[('GrandSlam', 'http://www.wikidata.org/entity/Q60874'), ('GrandSlamName', 'Australian Open')]
4


## Task 4
Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

In [7]:
# Q847 'tennis'
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   
   wd:Q847 ?p ?obj .
   
   ?p sc:name ?name.
   ?obj sc:name ?objName.
   filter regex(?objName, "court").
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('name', 'uses'), ('obj', 'http://www.wikidata.org/entity/Q741118'), ('objName', 'tennis court')]
1


In [10]:
#Q741118 'tennis court'
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   
   wd:Q741118 ?p ?obj .
   
   ?p sc:name ?name.
   ?obj sc:name ?objName.
   
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1535'), ('name', 'used by'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class'), ('obj', 'http://www.wikidata.org/entity/Q47677460'), ('objName', 'tennis playing surface')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q2310214'), ('objName', 'pitch')]
[('p', 'http://www.wikidata.org/prop/direct/P1535'), ('name', 'used by'), ('obj', 'http://www.wikidata.org/entity/Q1153025'), ('objName', 'wheelchair tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class'), ('obj', 'http://www.wikidata.org/entity/Q48291351'), ('objName', 'facet of tennis playing area')]
[('p', 'http://www.wikidat

In [11]:
#  Q47677460  'tennis playing surface'
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   
   wd:Q47677460 ?p ?obj .
   
   ?p sc:name ?name.
   ?obj sc:name ?objName.
   
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q741118'), ('objName', 'tennis court')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q28840755'), ('objName', 'surface')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q8842773'), ('objName', 'Category:Tennis court surfaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property'), ('obj', 'http://www.wikidata.org/entity/P765'), ('objName', 'surface played on')]
5


In [12]:
#P765  'surface played on'
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   
   wd:P765 ?p ?obj .
   
   ?p sc:name ?name.
   ?obj sc:name ?objName.
   
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('name', 'Wikidata property example'), ('obj', 'http://www.wikidata.org/entity/Q1613027'), ('objName', 'Stade Roland-Garros')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('name', 'Wikidata property example'), ('obj', 'http://www.wikidata.org/entity/Q119882'), ('objName', 'Palazzo dello Sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('name', 'Wikidata property example'), ('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('name', 'Wikidata item of this property'), ('obj', 'http://www.wikidata.org/entity/Q47677460'), ('objName', 'tennis playing surface')]
[('p', 'http://www.wikidata.org/prop/direct/P2302'), ('name', 'property constraint'), ('obj', 'http://www.wikidata.org/entity/Q21503250'), ('objName', 'type constraint')]
[('p', 'http://www.wikidata.org/prop/direct/P2302'), ('name', 'property constraint'), ('obj

In [15]:
#/Q119882  'Palazzo dello Sport'
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   
   wd:Q119882 ?p ?obj .
   
   ?p sc:name ?name.
   ?obj sc:name ?objName.
   
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P84'), ('name', 'architect'), ('obj', 'http://www.wikidata.org/entity/Q1382978'), ('objName', 'Marcello Piacentini')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity'), ('obj', 'http://www.wikidata.org/entity/Q16481947'), ('objName', 'Municipio IX')]
[('p', 'http://www.wikidata.org/prop/direct/P186'), ('name', 'made from material'), ('obj', 'http://www.wikidata.org/entity/Q184190'), ('objName', 'reinforced concrete')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q2033373'), ('objName', 'Palazzetto dello Sport')]
[('p', 'http://www.wikidata.org/prop/direct/P186'), ('name', 'made from material'), ('obj', 'http://www.wikidata.org/entity/Q2438447'), ('objName', 'ferrocement')]
[('p', 'http://www.wikidata.org/prop/direct/P466'), ('name', 'occupant'), ('obj', 'http://www.wikidata.org/entity/Q36462'), ('o

In [39]:
#P765'), ('name', 'surface played on'
queryString = """
SELECT DISTINCT  ?obj ?objName
WHERE {
   
   ?p wdt:P765 ?obj .
   
   ?p sc:name ?name.
   ?obj sc:name ?objName.
   
   
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q47677214'), ('objName', 'carpet')]
[('obj', 'http://www.wikidata.org/entity/Q133833'), ('objName', 'gravel')]
[('obj', 'http://www.wikidata.org/entity/Q13731184'), ('objName', 'artificial ice rink')]
[('obj', 'http://www.wikidata.org/entity/Q2306698'), ('objName', 'Desso GrassMaster')]
[('obj', 'http://www.wikidata.org/entity/Q3179760'), ('objName', 'tarmac')]
[('obj', 'http://www.wikidata.org/entity/Q190903'), ('objName', 'herbaceous plant')]
[('obj', 'http://www.wikidata.org/entity/Q198718'), ('objName', 'clay court')]
[('obj', 'http://www.wikidata.org/entity/Q838828'), ('objName', 'parquetry')]
[('obj', 'http://www.wikidata.org/entity/Q30198'), ('objName', 'marsh')]
[('obj', 'http://www.wikidata.org/entity/Q30688663'), ('objName', 'Mondotrack')]
10


Final query for this task

In [41]:
#Q47677214  'carpet'
queryString = """
SELECT DISTINCT ?p ?name  ?obj ?objName
WHERE {
   
   wd:Q47677214 ?p ?obj .
   
   ?p sc:name ?name.
   ?obj sc:name ?objName.
   
   
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1535'), ('name', 'used by'), ('obj', 'http://www.wikidata.org/entity/Q29167278'), ('objName', 'carpet court')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q47677460'), ('objName', 'tennis playing surface')]
3


In [56]:
# write your final query
# P279 'subclass of' Q47677460 'tennis playing surface' 
#P1535 'used by' Q29167278  'carpet court' 

queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   
   ?p wdt:P765 ?obj.
    ?obj  wdt:P279 wd:Q47677460.
    ?obj  wdt:P1535 ?court.
    
        
   ?p sc:name ?name.
   ?obj sc:name ?objName.
   #filter regex(?objName, "court").
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q47677214'), ('objName', 'carpet')]
[('obj', 'http://www.wikidata.org/entity/Q47676928'), ('objName', 'clay')]
[('obj', 'http://www.wikidata.org/entity/Q47678177'), ('objName', 'grass')]
[('obj', 'http://www.wikidata.org/entity/Q47777555'), ('objName', 'hardcourt surface')]
4


## Task 5
Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners (hint: men and women do not play together, there are two distinct tournament ) (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

In [59]:
# Q41520 Wimbledon 'Championships' 
#P31'), ('name', 'instance of')

queryString = """
SELECT DISTINCT ?p ?name 
WHERE { 
   ?p wdt:P31  wd:Q41520 .
   
   ?p sc:name ?name.
    
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q268871'), ('name', '2004 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/entity/Q173961'), ('name', '1978 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/entity/Q925026'), ('name', '1971 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/entity/Q15854765'), ('name', '2014 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/entity/Q30085309'), ('name', '2018 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/entity/Q745901'), ('name', '1974 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/entity/Q655368'), ('name', '1955 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/entity/Q655378'), ('name', '1956 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/entity/Q657950'), ('name', '1957 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/entity/Q657962'), ('name', '1958 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/entity/Q2296335'), ('name', '1881 Wi

In [60]:
# Q41520 Wimbledon 'Championships' 
#P31'), ('name', 'instance of')

queryString = """
SELECT DISTINCT ?p ?name 
WHERE { 
   ?p wdt:P31  wd:Q41520 .
   
   ?p sc:name ?name.
    filter regex(?name, "2018").
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q30085309'), ('name', '2018 Wimbledon Championships')]
1


In [62]:
#Q30085309 '2018 Wimbledon Championships' 
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE { 
  
   wd:Q30085309 ?p ?obj .
   
   ?p sc:name ?name.
   ?obj sc:name ?objName.
    
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q30098268'), ('objName', "2018 Wimbledon Championships – Women's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('obj', 'http://www.wikidata.org/entity/Q145'), ('objName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location'), ('obj', 'http://www.wikidata.org/entity/Q84'), ('objName', 'London')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q47345468'), ('objName', 'tennis tournament edition')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
[('p', 'http://www.wikidata.

In [65]:
#P527 'has part'
#Q30085309 '2018 Wimbledon Championships' 
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE { 
  
   wd:Q30085309 wdt:P527 ?obj .
   
    
   ?obj sc:name ?objName.
    
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q30098268'), ('objName', "2018 Wimbledon Championships – Women's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q32358401'), ('objName', '2018 Wimbledon Championships – Mixed Doubles')]
[('obj', 'http://www.wikidata.org/entity/Q32357790'), ('objName', "2018 Wimbledon Championships – Men's Doubles")]
[('obj', 'http://www.wikidata.org/entity/Q32356704'), ('objName', "2018 Wimbledon Championships – Women's Doubles")]
[('obj', 'http://www.wikidata.org/entity/Q55381839'), ('objName', "2018 Wimbledon Championships – Girls' Singles")]
[('obj', 'http://www.wikidata.org/entity/Q54961494'), ('objName', "2018 Wimbledon Championships – Men's Singles")]
[('obj', 'http://www.wikidata.org/entity/Q55410290'), ('objName', "2018 Wimbledon Championships – Girls' Doubles")]
[('obj', 'http://www.wikidata.org/entity/Q55410361'), ('objName', "2018 Wimbledon Championships – Boys' Doubles")]
[('obj', 'http://www.wikidata.org/entity/Q55455404'), ('objName', 

In [66]:
# Q30098268  "2018 Wimbledon Championships – Women's Singles" 
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE { 
  
   wd:Q30098268 ?p ?obj .
   
   ?p sc:name ?name.
   ?obj sc:name ?objName.
    
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q25344122'), ('objName', "2017 Wimbledon Championships – Women's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q30085309'), ('objName', '2018 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class'), ('obj', 'http://www.wikidata.org/entity/Q16893403'), ('objName', "women's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q46190676'), ('objName', 'tennis event')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q77178'), ('objName', 'Angelique Kerber')]
[('p', 

In [67]:
# write your final query
#P1346  'winner'
#P527 'has part'
#Q30085309 '2018 Wimbledon Championships' 
queryString = """
SELECT DISTINCT ?obj ?objName ?winner ?winnerName
WHERE { 
  
   wd:Q30085309 wdt:P527 ?obj.
   ?obj wdt:P1346 ?winner.
   ?obj sc:name ?objName.
   ?winner sc:name ?winnerName.
    
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q32357790'), ('objName', "2018 Wimbledon Championships – Men's Doubles"), ('winner', 'http://www.wikidata.org/entity/Q53342'), ('winnerName', 'Mike Bryan')]
[('obj', 'http://www.wikidata.org/entity/Q54961494'), ('objName', "2018 Wimbledon Championships – Men's Singles"), ('winner', 'http://www.wikidata.org/entity/Q5812'), ('winnerName', 'Novak Djokovic')]
[('obj', 'http://www.wikidata.org/entity/Q30098268'), ('objName', "2018 Wimbledon Championships – Women's Singles"), ('winner', 'http://www.wikidata.org/entity/Q77178'), ('winnerName', 'Angelique Kerber')]
[('obj', 'http://www.wikidata.org/entity/Q32356704'), ('objName', "2018 Wimbledon Championships – Women's Doubles"), ('winner', 'http://www.wikidata.org/entity/Q11052500'), ('winnerName', 'Barbora Krejčíková')]
[('obj', 'http://www.wikidata.org/entity/Q55455423'), ('objName', "2018 Wimbledon Championships – Wheelchair Women's Doubles"), ('winner', 'http://www.wikidata.org/entity/Q1622

## Task 6
Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

In [73]:
#Q10833314 'tennis player' 
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE { 
  
    ?p ?obj wd:Q10833314.
   
   ?p sc:name ?name.
   ?obj sc:name ?objName.
    
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q6053701'), ('name', 'Template:Infobox tennis biography'), ('obj', 'http://www.wikidata.org/prop/direct/P1423'), ('objName', 'template has topic')]
[('p', 'http://www.wikidata.org/entity/Q1683632'), ('name', 'Jaroslav Navrátil'), ('obj', 'http://www.wikidata.org/prop/direct/P106'), ('objName', 'occupation')]
[('p', 'http://www.wikidata.org/entity/Q138094'), ('name', 'Anna Karolína Schmiedlová'), ('obj', 'http://www.wikidata.org/prop/direct/P106'), ('objName', 'occupation')]
[('p', 'http://www.wikidata.org/entity/Q15197746'), ('name', 'Wesley Koolhof'), ('obj', 'http://www.wikidata.org/prop/direct/P106'), ('objName', 'occupation')]
[('p', 'http://www.wikidata.org/entity/Q1558013'), ('name', 'Pierre Barthès'), ('obj', 'http://www.wikidata.org/prop/direct/P106'), ('objName', 'occupation')]
[('p', 'http://www.wikidata.org/entity/Q15849678'), ('name', 'Sue Partridge'), ('obj', 'http://www.wikidata.org/prop/direct/P106'), ('objName', 'occupation

In [75]:
#P106  'occupation' 
#Q10833314 'tennis player' 
queryString = """
SELECT DISTINCT ?p ?name 
WHERE { 
  
    ?p wdt:P106 wd:Q10833314.
   
   ?p sc:name ?name.
    
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1683632'), ('name', 'Jaroslav Navrátil')]
[('p', 'http://www.wikidata.org/entity/Q138094'), ('name', 'Anna Karolína Schmiedlová')]
[('p', 'http://www.wikidata.org/entity/Q15197746'), ('name', 'Wesley Koolhof')]
[('p', 'http://www.wikidata.org/entity/Q1558013'), ('name', 'Pierre Barthès')]
[('p', 'http://www.wikidata.org/entity/Q15849678'), ('name', 'Sue Partridge')]
[('p', 'http://www.wikidata.org/entity/Q15880574'), ('name', 'Paul Oosterbaan')]
[('p', 'http://www.wikidata.org/entity/Q240837'), ('name', 'Rika Fujiwara')]
[('p', 'http://www.wikidata.org/entity/Q4134097'), ('name', 'Margarita Gasparyan')]
[('p', 'http://www.wikidata.org/entity/Q589611'), ('name', 'Philippe Chatrier')]
[('p', 'http://www.wikidata.org/entity/Q16009513'), ('name', 'James Bayley')]
10


In [79]:
#Q1683632'), ('name', 'Jaroslav Navrátil')]
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE { 
  
    wd:Q1683632 ?p ?obj.
   
   ?p sc:name ?name.
   optional{?obj sc:name ?objName.}
    
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q13639102'), ('objName', 'Jaroslav Navrátil')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('name', 'given name'), ('obj', 'http://www.wikidata.org/entity/Q35000543'), ('objName', 'Jaroslav')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth'), ('obj', 'http://www.wikidata.org/entity/Q470380'), ('objName', 'Přerov')]
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('name', 'family name'), ('obj', 'http://www.wikidata.org/entity/Q21492186'), ('objName', 'Navrátil')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation'), ('obj', 'http://www.wikidata.org/entity/Q10833314'), ('objName', 'tennis player')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport'), ('obj', 'http://www.wikidata.org/entity/Q213'), ('objName', 'Czech Republic')]
[('p', 'http://www.wikidata.org/prop/dir

In [63]:
#P569 'date of birth'
#P106  'occupation' 
#Q10833314 'tennis player' 

queryString = """
SELECT DISTINCT ?p ?name ?dateBirthday
WHERE { 
  
    ?p wdt:P106 wd:Q10833314;
        wdt:P569 ?dateBirthday.
    filter (?dateBirthday > "1975-01-01T10:00:00"^^xsd:dateTime).
    
   ?p sc:name ?name.
    
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q138094'), ('name', 'Anna Karolína Schmiedlová'), ('dateBirthday', '1994-09-13T00:00:00Z')]
[('p', 'http://www.wikidata.org/entity/Q15197746'), ('name', 'Wesley Koolhof'), ('dateBirthday', '1989-04-17T00:00:00Z')]
[('p', 'http://www.wikidata.org/entity/Q15880574'), ('name', 'Paul Oosterbaan'), ('dateBirthday', '1995-07-08T00:00:00Z')]
[('p', 'http://www.wikidata.org/entity/Q240837'), ('name', 'Rika Fujiwara'), ('dateBirthday', '1981-09-19T00:00:00Z')]
[('p', 'http://www.wikidata.org/entity/Q4134097'), ('name', 'Margarita Gasparyan'), ('dateBirthday', '1994-09-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/entity/Q16224631'), ('name', 'Lee So-ra'), ('dateBirthday', '1994-07-22T00:00:00Z')]
[('p', 'http://www.wikidata.org/entity/Q7281705'), ('name', 'Radu Albot'), ('dateBirthday', '1989-11-11T00:00:00Z')]
[('p', 'http://www.wikidata.org/entity/Q18918709'), ('name', 'Markéta Vondroušová'), ('dateBirthday', '1999-06-28T00:00:00Z')]
[('p', 'ht

In [51]:
# Q102113  'Grand Slam of tennis' 
# P31  'instance of'
queryString = """
SELECT DISTINCT ?GrandSlam ?GrandSlamName  ?edition ?editionName
WHERE { 
    ?GrandSlam wdt:P361 wd:Q102113.
    ?edition ?p ?GrandSlam.
   
   ?GrandSlam sc:name ?GrandSlamName.
   ?edition sc:name ?editionName.
   
    
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('GrandSlam', 'http://www.wikidata.org/entity/Q41520'), ('GrandSlamName', 'Wimbledon Championships'), ('edition', 'http://www.wikidata.org/entity/Q268871'), ('editionName', '2004 Wimbledon Championships')]
[('GrandSlam', 'http://www.wikidata.org/entity/Q60874'), ('GrandSlamName', 'Australian Open'), ('edition', 'http://www.wikidata.org/entity/Q602116'), ('editionName', '1969 Australian Open')]
[('GrandSlam', 'http://www.wikidata.org/entity/Q123577'), ('GrandSlamName', 'US Open'), ('edition', 'http://www.wikidata.org/entity/Q643498'), ('editionName', '2000 US Open')]
[('GrandSlam', 'http://www.wikidata.org/entity/Q43605'), ('GrandSlamName', 'French Open'), ('edition', 'http://www.wikidata.org/entity/Q660527'), ('editionName', '1983 French Open')]
[('GrandSlam', 'http://www.wikidata.org/entity/Q123577'), ('GrandSlamName', 'US Open'), ('edition', 'http://www.wikidata.org/entity/Q840907'), ('editionName', '2004 US Open')]
[('GrandSlam', 'http://www.wikidata.org/entity/Q43605'), ('

In [65]:
# Q268871  '2004 Wimbledon Championships'
# P31  'instance of'
queryString = """
SELECT DISTINCT ?p ?name  ?obj ?objName
WHERE { 
    wd:Q268871 ?p ?obj. 
   
   ?obj sc:name ?objName.
   ?p sc:name ?name.

    
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q832484'), ('objName', '2003 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by'), ('obj', 'http://www.wikidata.org/entity/Q746693'), ('objName', '2005 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('obj', 'http://www.wikidata.org/entity/Q145'), ('objName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q47345468'), ('objName', 'tennis tournament edition')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q41520'), ('objName', 'Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q1575133'), ('objName', "2004 Wimbledon Champion

In [67]:
# Q102113  'Grand Slam of tennis'
# P527  'has part'
# P31  'instance of'
queryString = """
SELECT DISTINCT ?p ?name 
WHERE { 
    ?GrandSlam wdt:P361 wd:Q102113.
    ?edition wdt:P31 ?GrandSlam.
    ?edition wdt:P527 ?p.
    
   ?p sc:name ?name.

    
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q15854755'), ('name', "1978 Wimbledon Championships – Men's Doubles")]
[('p', 'http://www.wikidata.org/entity/Q20895284'), ('name', '2015 US Open – Mixed Doubles')]
[('p', 'http://www.wikidata.org/entity/Q15854758'), ('name', "1982 Wimbledon Championships – Men's Doubles")]
[('p', 'http://www.wikidata.org/entity/Q22231236'), ('name', "2016 Australian Open – Boys' Doubles")]
[('p', 'http://www.wikidata.org/entity/Q22956224'), ('name', "2017 Australian Open – Men's Singles")]
[('p', 'http://www.wikidata.org/entity/Q25344122'), ('name', "2017 Wimbledon Championships – Women's Singles")]
[('p', 'http://www.wikidata.org/entity/Q30098268'), ('name', "2018 Wimbledon Championships – Women's Singles")]
[('p', 'http://www.wikidata.org/entity/Q1455356'), ('name', "1980 French Open – Women's Doubles")]
[('p', 'http://www.wikidata.org/entity/Q1751593'), ('name', "1958 Australian Championships – Men's Singles")]
[('p', 'http://www.wikidata.org/entity/Q2

In [68]:

# Q15854755 "1978 Wimbledon Championships – Men's Doubles"
# P527  'has part'
# P31  'instance of'
queryString = """
SELECT DISTINCT ?p ?name  ?obj ?objName
WHERE { 
    wd:Q15854755 ?p ?obj. 
   
   ?obj sc:name ?objName.
   ?p sc:name ?name.

    
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q52453'), ('objName', 'Bob Hewitt')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class'), ('obj', 'http://www.wikidata.org/entity/Q17299700'), ('objName', "men's doubles")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q46190676'), ('objName', 'tennis event')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q173961'), ('objName', '1978 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('objName', 'tennis')]
5


In [70]:
# P1346  'winner'
# Q102113  'Grand Slam of tennis'
# P527  'has part'
# P31  'instance of'
queryString = """
SELECT DISTINCT ?subEdition ?subEditionName ?winner ?winnerName
WHERE { 
    ?GrandSlam wdt:P361 wd:Q102113.
    ?edition wdt:P31 ?GrandSlam.
    
    ?edition wdt:P527 ?subEdition.
    ?subEdition wdt:P1346 ?winner.
    
   ?subEdition sc:name ?subEditionName.
    ?winner sc:name ?winnerName.
    
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)


Results
[('subEdition', 'http://www.wikidata.org/entity/Q15854755'), ('subEditionName', "1978 Wimbledon Championships – Men's Doubles"), ('winner', 'http://www.wikidata.org/entity/Q52453'), ('winnerName', 'Bob Hewitt')]
[('subEdition', 'http://www.wikidata.org/entity/Q20895284'), ('subEditionName', '2015 US Open – Mixed Doubles'), ('winner', 'http://www.wikidata.org/entity/Q134720'), ('winnerName', 'Martina Hingis')]
[('subEdition', 'http://www.wikidata.org/entity/Q20895284'), ('subEditionName', '2015 US Open – Mixed Doubles'), ('winner', 'http://www.wikidata.org/entity/Q296282'), ('winnerName', 'Leander Paes')]
[('subEdition', 'http://www.wikidata.org/entity/Q15854758'), ('subEditionName', "1982 Wimbledon Championships – Men's Doubles"), ('winner', 'http://www.wikidata.org/entity/Q1348791'), ('winnerName', 'Paul McNamee')]
[('subEdition', 'http://www.wikidata.org/entity/Q15854758'), ('subEditionName', "1982 Wimbledon Championships – Men's Doubles"), ('winner', 'http://www.wikidata.org

In [75]:
# P1346  'winner'
# Q102113  'Grand Slam of tennis'
# P527  'has part'
# P31  'instance of'
queryString = """
SELECT DISTINCT ?winner ?winnerName (count(?subEdition) as ?numWin) 
WHERE { 
    ?GrandSlam wdt:P361 wd:Q102113.
    ?edition wdt:P31 ?GrandSlam.
    ?edition wdt:P527 ?subEdition.
    ?subEdition wdt:P1346 ?winner.
    
   ?subEdition sc:name ?subEditionName.
    ?winner sc:name ?winnerName.
    
}

group by ?winner ?winnerName

order by DESC (?numWin)
LIMIT 10
"""

print("Results")
x=run_query(queryString)



Results
[('winner', 'http://www.wikidata.org/entity/Q54545'), ('winnerName', 'Martina Navratilova'), ('numWin', '49')]
[('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('numWin', '36')]
[('winner', 'http://www.wikidata.org/entity/Q202342'), ('winnerName', 'Margaret Court'), ('numWin', '31')]
[('winner', 'http://www.wikidata.org/entity/Q134720'), ('winnerName', 'Martina Hingis'), ('numWin', '23')]
[('winner', 'http://www.wikidata.org/entity/Q11662'), ('winnerName', 'Steffi Graf'), ('numWin', '21')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 'Venus Williams'), ('numWin', '19')]
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', 'Rafael Nadal'), ('numWin', '19')]
[('winner', 'http://www.wikidata.org/entity/Q5812'), ('winnerName', 'Novak Djokovic'), ('numWin', '19')]
[('winner', 'http://www.wikidata.org/entity/Q1426'), ('winnerName', 'Roger Federer'), ('numWin', '17')]
[('winner', 'http://www.wikidata.org/e

In [77]:

# P1346  'winner'
# Q102113  'Grand Slam of tennis'
# P527  'has part'
# P31  'instance of'
queryString = """
SELECT DISTINCT ?winner ?winnerName (count(?subEdition) as ?numWin) 
WHERE { 
    ?GrandSlam wdt:P361 wd:Q102113.
    ?edition wdt:P31 ?GrandSlam.
    
    ?edition wdt:P527 ?subEdition.
    ?subEdition wdt:P1346 ?winner.
    
    ?winner wdt:P569 ?dateBirthday.
    filter (?dateBirthday >= "1975-01-01T00:00:00"^^xsd:dateTime).
    
   ?subEdition sc:name ?subEditionName.
    ?winner sc:name ?winnerName.
    
}

group by ?winner ?winnerName

order by DESC (?numWin)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('numWin', '36')]
[('winner', 'http://www.wikidata.org/entity/Q134720'), ('winnerName', 'Martina Hingis'), ('numWin', '23')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 'Venus Williams'), ('numWin', '19')]
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', 'Rafael Nadal'), ('numWin', '19')]
[('winner', 'http://www.wikidata.org/entity/Q5812'), ('winnerName', 'Novak Djokovic'), ('numWin', '19')]
[('winner', 'http://www.wikidata.org/entity/Q1426'), ('winnerName', 'Roger Federer'), ('numWin', '17')]
[('winner', 'http://www.wikidata.org/entity/Q24574661'), ('winnerName', 'Diede de Groot'), ('numWin', '16')]
[('winner', 'http://www.wikidata.org/entity/Q53340'), ('winnerName', 'Bob Bryan'), ('numWin', '14')]
[('winner', 'http://www.wikidata.org/entity/Q16224602'), ('winnerName', 'Yui Kamiji'), ('numWin', '12')]
[('winner', 'http://www.wikidata.org/entity/

In [79]:

# P1346  'winner'
# Q102113  'Grand Slam of tennis'
# P527  'has part'
# P31  'instance of'
queryString = """
SELECT DISTINCT  ?winner ?winnerName (count(?subEdition) as ?numWin) 
WHERE { 
    ?GrandSlam wdt:P361 wd:Q102113.
    ?edition wdt:P31 ?GrandSlam.
    
    ?subEdition wdt:P361 ?edition.
    ?subEdition wdt:P1346 ?winner.
    
    ?winner wdt:P569 ?dateBirthday.
    filter (?dateBirthday >= "1975-01-01T00:00:00"^^xsd:dateTime).
    
   ?subEdition sc:name ?subEditionName.
    ?winner sc:name ?winnerName.
    
}

group by  ?winner ?winnerName

order by DESC (?numWin)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q11459'), ('winnerName', 'Serena Williams'), ('numWin', '39')]
[('winner', 'http://www.wikidata.org/entity/Q959958'), ('winnerName', 'Shingo Kunieda'), ('numWin', '32')]
[('winner', 'http://www.wikidata.org/entity/Q134720'), ('winnerName', 'Martina Hingis'), ('numWin', '25')]
[('winner', 'http://www.wikidata.org/entity/Q16224602'), ('winnerName', 'Yui Kamiji'), ('numWin', '22')]
[('winner', 'http://www.wikidata.org/entity/Q1426'), ('winnerName', 'Roger Federer'), ('numWin', '22')]
[('winner', 'http://www.wikidata.org/entity/Q259606'), ('winnerName', 'Esther Vergeer'), ('numWin', '21')]
[('winner', 'http://www.wikidata.org/entity/Q11578'), ('winnerName', 'Venus Williams'), ('numWin', '20')]
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', 'Rafael Nadal'), ('numWin', '20')]
[('winner', 'http://www.wikidata.org/entity/Q5812'), ('winnerName', 'Novak Djokovic'), ('numWin', '20')]
[('winner', 'http://www.wikidata.org/ent

We have 2 possibile way to return the list with differente result. We chooese the last one the more complite of information

## Task 7
Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

In [108]:

# P1346  'winner'
# Q102113  'Grand Slam of tennis'
# P527  'has part'
# P31  'instance of'
#[('GrandSlam', 'http://www.wikidata.org/entity/Q43605'), ('GrandSlamName', 'French Open')]
#[('GrandSlam', 'http://www.wikidata.org/entity/Q123577'), ('GrandSlamName', 'US Open')]
#[('GrandSlam', 'http://www.wikidata.org/entity/Q41520'), ('GrandSlamName', 'Wimbledon Championships')]
#[('GrandSlam', 'http://www.wikidata.org/entity/Q60874'), ('GrandSlamName', 'Australian Open')]
queryString = """
SELECT DISTINCT  ?winner ?winnerName (count(?subEdition) as ?numWinFrenchOpen)  ?numWinUSOpen
WHERE { 
    
    ?edition wdt:P31 wd:Q43605.
    
    ?subEdition wdt:P361 ?edition.
    ?subEdition wdt:P1346 ?winner.
    
    {
        SELECT DISTINCT  ?winner (count(?subEdition2) as ?numWinUSOpen) 
        WHERE { 

            ?edition2 wdt:P31 wd:Q123577.

            ?subEdition2 wdt:P361 ?edition2.
            ?subEdition2 wdt:P1346 ?winner.
        }
        group by  ?winner
        having (count(?subEdition2) as ?numWinUSOpen > 1)
         
        LIMIT 100
    }

    ?winner sc:name ?winnerName.
    
}

group by  ?winner ?winnerName ?numWinUSOpen
having (count(?subEdition) as ?numWinFrenchOpen > 1)
order by DESC (?numWinFrenchOpen)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', 'Rafael Nadal'), ('numWinFrenchOpen', '13'), ('numWinUSOpen', '4')]
[('winner', 'http://www.wikidata.org/entity/Q202342'), ('winnerName', 'Margaret Court'), ('numWinFrenchOpen', '12'), ('numWinUSOpen', '10')]
[('winner', 'http://www.wikidata.org/entity/Q959958'), ('winnerName', 'Shingo Kunieda'), ('numWinFrenchOpen', '10'), ('numWinUSOpen', '7')]
[('winner', 'http://www.wikidata.org/entity/Q14286'), ('winnerName', 'Chris Evert'), ('numWinFrenchOpen', '9'), ('numWinUSOpen', '6')]
[('winner', 'http://www.wikidata.org/entity/Q3042015'), ('winnerName', 'Stéphane Houdet'), ('numWinFrenchOpen', '7'), ('numWinUSOpen', '6')]
[('winner', 'http://www.wikidata.org/entity/Q11662'), ('winnerName', 'Steffi Graf'), ('numWinFrenchOpen', '6'), ('numWinUSOpen', '5')]
[('winner', 'http://www.wikidata.org/entity/Q259606'), ('winnerName', 'Esther Vergeer'), ('numWinFrenchOpen', '6'), ('numWinUSOpen', '7')]
[('winner', 'http://www.

In [113]:

# P1346  'winner'
# Q102113  'Grand Slam of tennis'
# P527  'has part'
# P31  'instance of'
#[('GrandSlam', 'http://www.wikidata.org/entity/Q43605'), ('GrandSlamName', 'French Open')]
#[('GrandSlam', 'http://www.wikidata.org/entity/Q123577'), ('GrandSlamName', 'US Open')]
#[('GrandSlam', 'http://www.wikidata.org/entity/Q41520'), ('GrandSlamName', 'Wimbledon Championships')]
#[('GrandSlam', 'http://www.wikidata.org/entity/Q60874'), ('GrandSlamName', 'Australian Open')]
queryString = """
SELECT DISTINCT  ?winner ?winnerName (count(?subEdition) as ?numWinFrenchOpen)  ?numWinUSOpen ?numWinWimbledon
WHERE { 
    
    ?edition wdt:P31 wd:Q43605.
    
    ?subEdition wdt:P361 ?edition.
    ?subEdition wdt:P1346 ?winner.
    
    {
        SELECT DISTINCT  ?winner (count(?subEdition2) as ?numWinUSOpen) ?numWinWimbledon
        WHERE { 

            ?edition2 wdt:P31 wd:Q123577.

            ?subEdition2 wdt:P361 ?edition2.
            ?subEdition2 wdt:P1346 ?winner.
            
            {
                SELECT DISTINCT  ?winner (count(?subEdition3) as ?numWinWimbledon) 
                WHERE { 

                    ?edition3 wdt:P31 wd:Q41520.

                    ?subEdition3 wdt:P361 ?edition3.
                    ?subEdition3 wdt:P1346 ?winner.
                }
                group by  ?winner
                having (count(?subEdition3) as ?numWinWimbledon > 1)

                #LIMIT 100
            
            }
        }
        group by  ?winner ?numWinWimbledon
        having (count(?subEdition2) as ?numWinUSOpen > 1)
         
        #LIMIT 100
    }

    ?winner sc:name ?winnerName.
    
}

group by  ?winner ?winnerName ?numWinUSOpen ?numWinWimbledon
having (count(?subEdition) as ?numWinFrenchOpen > 1)
order by DESC (?numWinFrenchOpen)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q10132'), ('winnerName', 'Rafael Nadal'), ('numWinFrenchOpen', '13'), ('numWinUSOpen', '4'), ('numWinWimbledon', '2')]
[('winner', 'http://www.wikidata.org/entity/Q202342'), ('winnerName', 'Margaret Court'), ('numWinFrenchOpen', '12'), ('numWinUSOpen', '10'), ('numWinWimbledon', '5')]
[('winner', 'http://www.wikidata.org/entity/Q54545'), ('winnerName', 'Martina Navratilova'), ('numWinFrenchOpen', '11'), ('numWinUSOpen', '13'), ('numWinWimbledon', '22')]
[('winner', 'http://www.wikidata.org/entity/Q14286'), ('winnerName', 'Chris Evert'), ('numWinFrenchOpen', '9'), ('numWinUSOpen', '6'), ('numWinWimbledon', '4')]
[('winner', 'http://www.wikidata.org/entity/Q16224602'), ('winnerName', 'Yui Kamiji'), ('numWinFrenchOpen', '7'), ('numWinUSOpen', '4'), ('numWinWimbledon', '6')]
[('winner', 'http://www.wikidata.org/entity/Q3042015'), ('winnerName', 'Stéphane Houdet'), ('numWinFrenchOpen', '7'), ('numWinUSOpen', '6'), ('numWinWimbledon', '3')]

In [116]:

# P1346  'winner'
# Q102113  'Grand Slam of tennis'
# P527  'has part'
# P31  'instance of'
#[('GrandSlam', 'http://www.wikidata.org/entity/Q43605'), ('GrandSlamName', 'French Open')]
#[('GrandSlam', 'http://www.wikidata.org/entity/Q123577'), ('GrandSlamName', 'US Open')]
#[('GrandSlam', 'http://www.wikidata.org/entity/Q41520'), ('GrandSlamName', 'Wimbledon Championships')]
#[('GrandSlam', 'http://www.wikidata.org/entity/Q60874'), ('GrandSlamName', 'Australian Open')]
queryString = """
SELECT DISTINCT  ?winner ?winnerName (count(?subEdition) as ?numWinFrenchOpen)  ?numWinUSOpen ?numWinWimbledon ?numWinAustralian
WHERE { 
    
    ?edition wdt:P31 wd:Q43605.
    
    ?subEdition wdt:P361 ?edition.
    ?subEdition wdt:P1346 ?winner.
    
    { #SUBQUERY 2
        SELECT DISTINCT  ?winner (count(?subEdition2) as ?numWinUSOpen) ?numWinWimbledon ?numWinAustralian
        WHERE { 

            ?edition2 wdt:P31 wd:Q123577.

            ?subEdition2 wdt:P361 ?edition2.
            ?subEdition2 wdt:P1346 ?winner.
            
            { #SUBQUERY 3
                SELECT DISTINCT  ?winner (count(?subEdition3) as ?numWinWimbledon) ?numWinAustralian
                WHERE { 

                    ?edition3 wdt:P31 wd:Q41520.

                    ?subEdition3 wdt:P361 ?edition3.
                    ?subEdition3 wdt:P1346 ?winner.
                    
                    { #SUBQUERY 4
                        SELECT DISTINCT  ?winner (count(?subEdition4) as ?numWinAustralian) 
                        WHERE { 

                            ?edition4 wdt:P31 wd:Q60874.

                            ?subEdition4 wdt:P361 ?edition4.
                            ?subEdition4 wdt:P1346 ?winner.
                        }
                        group by  ?winner
                        having (count(?subEdition4) as ?numWinUSOpen > 1)

                    }
                    
                    
                }
                group by  ?winner ?numWinAustralian
                having (count(?subEdition3) as ?numWinWimbledon > 1) 

                
            
            }
        }
        group by  ?winner ?numWinWimbledon ?numWinAustralian
        having (count(?subEdition2) as ?numWinUSOpen > 1)
         
       
    }

    ?winner sc:name ?winnerName.
    
}

group by  ?winner ?winnerName ?numWinUSOpen ?numWinWimbledon ?numWinAustralian
having (count(?subEdition) as ?numWinFrenchOpen > 1)
order by DESC (?numWinFrenchOpen)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q202342'), ('winnerName', 'Margaret Court'), ('numWinFrenchOpen', '12'), ('numWinUSOpen', '10'), ('numWinWimbledon', '5'), ('numWinAustralian', '21')]
[('winner', 'http://www.wikidata.org/entity/Q54545'), ('winnerName', 'Martina Navratilova'), ('numWinFrenchOpen', '11'), ('numWinUSOpen', '13'), ('numWinWimbledon', '22'), ('numWinAustralian', '12')]
[('winner', 'http://www.wikidata.org/entity/Q14286'), ('winnerName', 'Chris Evert'), ('numWinFrenchOpen', '9'), ('numWinUSOpen', '6'), ('numWinWimbledon', '4'), ('numWinAustralian', '2')]
[('winner', 'http://www.wikidata.org/entity/Q3042015'), ('winnerName', 'Stéphane Houdet'), ('numWinFrenchOpen', '7'), ('numWinUSOpen', '6'), ('numWinWimbledon', '3'), ('numWinAustralian', '5')]
[('winner', 'http://www.wikidata.org/entity/Q16224602'), ('winnerName', 'Yui Kamiji'), ('numWinFrenchOpen', '7'), ('numWinUSOpen', '4'), ('numWinWimbledon', '6'), ('numWinAustralian', '5')]
[('winner', 'http://www.w

In [118]:

# P1346  'winner'
# Q102113  'Grand Slam of tennis'
# P527  'has part'
# P31  'instance of'
#[('GrandSlam', 'http://www.wikidata.org/entity/Q43605'), ('GrandSlamName', 'French Open')]
#[('GrandSlam', 'http://www.wikidata.org/entity/Q123577'), ('GrandSlamName', 'US Open')]
#[('GrandSlam', 'http://www.wikidata.org/entity/Q41520'), ('GrandSlamName', 'Wimbledon Championships')]
#[('GrandSlam', 'http://www.wikidata.org/entity/Q60874'), ('GrandSlamName', 'Australian Open')]

# P1532  'country for sport'
queryString = """
 SELECT DISTINCT  ?winner ?winnerName ?countryName
WHERE { 
    ?winner wdt:P1532 ?country.
    ?country sc:name ?countryName.
    
        {

        SELECT DISTINCT  ?winner ?winnerName (count(?subEdition) as ?numWinFrenchOpen)  ?numWinUSOpen ?numWinWimbledon ?numWinAustralian
        WHERE { 

            ?edition wdt:P31 wd:Q43605.

            ?subEdition wdt:P361 ?edition.
            ?subEdition wdt:P1346 ?winner.

            { #SUBQUERY 2
                SELECT DISTINCT  ?winner (count(?subEdition2) as ?numWinUSOpen) ?numWinWimbledon ?numWinAustralian
                WHERE { 

                    ?edition2 wdt:P31 wd:Q123577.

                    ?subEdition2 wdt:P361 ?edition2.
                    ?subEdition2 wdt:P1346 ?winner.

                    { #SUBQUERY 3
                        SELECT DISTINCT  ?winner (count(?subEdition3) as ?numWinWimbledon) ?numWinAustralian
                        WHERE { 

                            ?edition3 wdt:P31 wd:Q41520.

                            ?subEdition3 wdt:P361 ?edition3.
                            ?subEdition3 wdt:P1346 ?winner.

                            { #SUBQUERY 4
                                SELECT DISTINCT  ?winner (count(?subEdition4) as ?numWinAustralian) 
                                WHERE { 

                                    ?edition4 wdt:P31 wd:Q60874.

                                    ?subEdition4 wdt:P361 ?edition4.
                                    ?subEdition4 wdt:P1346 ?winner.
                                }
                                group by  ?winner
                                having (count(?subEdition4) as ?numWinUSOpen > 1)

                            }


                        }
                        group by  ?winner ?numWinAustralian
                        having (count(?subEdition3) as ?numWinWimbledon > 1) 



                    }
                }
                group by  ?winner ?numWinWimbledon ?numWinAustralian
                having (count(?subEdition2) as ?numWinUSOpen > 1)


            }

            ?winner sc:name ?winnerName.

        }

        group by  ?winner ?winnerName ?numWinUSOpen ?numWinWimbledon ?numWinAustralian
        having (count(?subEdition) as ?numWinFrenchOpen > 1)
        order by DESC (?numWinFrenchOpen)
        LIMIT 100
        }
}
"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q3042015'), ('winnerName', 'Stéphane Houdet'), ('countryName', 'France')]
[('winner', 'http://www.wikidata.org/entity/Q16224602'), ('winnerName', 'Yui Kamiji'), ('countryName', 'Japan')]
[('winner', 'http://www.wikidata.org/entity/Q11662'), ('winnerName', 'Steffi Graf'), ('countryName', 'Germany')]
[('winner', 'http://www.wikidata.org/entity/Q233243'), ('winnerName', 'Natasha Zvereva'), ('countryName', 'Belarus')]
[('winner', 'http://www.wikidata.org/entity/Q54545'), ('winnerName', 'Martina Navratilova'), ('countryName', 'Czech Republic')]
[('winner', 'http://www.wikidata.org/entity/Q229121'), ('winnerName', 'Jana Novotná'), ('countryName', 'Czech Republic')]
[('winner', 'http://www.wikidata.org/entity/Q54654'), ('winnerName', 'Pam Shriver'), ('countryName', 'United States of America')]
[('winner', 'http://www.wikidata.org/entity/Q14286'), ('winnerName', 'Chris Evert'), ('countryName', 'United States of America')]
[('winner', 'http://

Final query for this task

In [119]:

# P1346  'winner'
# Q102113  'Grand Slam of tennis'
# P527  'has part'
# P31  'instance of'
#[('GrandSlam', 'http://www.wikidata.org/entity/Q43605'), ('GrandSlamName', 'French Open')]
#[('GrandSlam', 'http://www.wikidata.org/entity/Q123577'), ('GrandSlamName', 'US Open')]
#[('GrandSlam', 'http://www.wikidata.org/entity/Q41520'), ('GrandSlamName', 'Wimbledon Championships')]
#[('GrandSlam', 'http://www.wikidata.org/entity/Q60874'), ('GrandSlamName', 'Australian Open')]

# P1532  'country for sport'
queryString = """
 SELECT DISTINCT  ?country ?countryName (count(?winner)as ?numWin)
WHERE { 
    ?winner wdt:P1532 ?country.
    ?country sc:name ?countryName.
    
        {

        SELECT DISTINCT  ?winner ?winnerName (count(?subEdition) as ?numWinFrenchOpen)  ?numWinUSOpen ?numWinWimbledon ?numWinAustralian
        WHERE { 

            ?edition wdt:P31 wd:Q43605.

            ?subEdition wdt:P361 ?edition.
            ?subEdition wdt:P1346 ?winner.

            { #SUBQUERY 2
                SELECT DISTINCT  ?winner (count(?subEdition2) as ?numWinUSOpen) ?numWinWimbledon ?numWinAustralian
                WHERE { 

                    ?edition2 wdt:P31 wd:Q123577.

                    ?subEdition2 wdt:P361 ?edition2.
                    ?subEdition2 wdt:P1346 ?winner.

                    { #SUBQUERY 3
                        SELECT DISTINCT  ?winner (count(?subEdition3) as ?numWinWimbledon) ?numWinAustralian
                        WHERE { 

                            ?edition3 wdt:P31 wd:Q41520.

                            ?subEdition3 wdt:P361 ?edition3.
                            ?subEdition3 wdt:P1346 ?winner.

                            { #SUBQUERY 4
                                SELECT DISTINCT  ?winner (count(?subEdition4) as ?numWinAustralian) 
                                WHERE { 

                                    ?edition4 wdt:P31 wd:Q60874.

                                    ?subEdition4 wdt:P361 ?edition4.
                                    ?subEdition4 wdt:P1346 ?winner.
                                }
                                group by  ?winner
                                having (count(?subEdition4) as ?numWinUSOpen > 1)

                            }


                        }
                        group by  ?winner ?numWinAustralian
                        having (count(?subEdition3) as ?numWinWimbledon > 1) 



                    }
                }
                group by  ?winner ?numWinWimbledon ?numWinAustralian
                having (count(?subEdition2) as ?numWinUSOpen > 1)


            }

            ?winner sc:name ?winnerName.

        }

        group by  ?winner ?winnerName ?numWinUSOpen ?numWinWimbledon ?numWinAustralian
        having (count(?subEdition) as ?numWinFrenchOpen > 1)
        order by DESC (?numWinFrenchOpen)
        LIMIT 100
        }
}
GROUP BY ?country ?countryName

"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numWin', '1')]
[('country', 'http://www.wikidata.org/entity/Q403'), ('countryName', 'Serbia'), ('numWin', '1')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('numWin', '1')]
[('country', 'http://www.wikidata.org/entity/Q184'), ('countryName', 'Belarus'), ('numWin', '1')]
[('country', 'http://www.wikidata.org/entity/Q668'), ('countryName', 'India'), ('numWin', '1')]
[('country', 'http://www.wikidata.org/entity/Q213'), ('countryName', 'Czech Republic'), ('numWin', '2')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('countryName', 'Japan'), ('numWin', '1')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America'), ('numWin', '9')]
[('country', 'http://www.wikidata.org/entity/Q408'), ('countryName', 'Australia'), ('numWin', '1')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('countryName', 'Switzerland'), ('numWin

## Task 8
Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

In [35]:
# P1532  'country for sport' property for the player
# P641 'sport' Q847  'tennis' 

queryString = """
SELECT DISTINCT ?obj ?objName 
WHERE { 
  
   ?obj ?r  wd:Q847.
        
    
   ?obj sc:name ?objName.
   filter regex(?objName, "ranking").
   
    
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q50932406'), ('objName', 'tennis ranking')]
[('obj', 'http://www.wikidata.org/entity/Q686735'), ('objName', 'World number 1 male tennis player rankings')]
2


In [38]:
# Q50932406 'tennis ranking' 
# P1532  'country for sport' property for the player
# P641 'sport' Q847  'tennis' 

queryString = """
SELECT DISTINCT ?obj ?objName 
WHERE { 
  
   ?obj ?r wd:Q50932406.
        
    
   ?obj sc:name ?objName.
   
    
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q15207863'), ('objName', 'Category:Tennis rankings')]
[('obj', 'http://www.wikidata.org/entity/Q56377500'), ('objName', 'Transition Tour')]
[('obj', 'http://www.wikidata.org/entity/Q10272773'), ('objName', 'WTA Rankings')]
[('obj', 'http://www.wikidata.org/entity/Q1571500'), ('objName', 'ATP Rankings')]
4


In [120]:
# Q50932406 'tennis ranking' 
# P1532  'country for sport' property for the player
# P641 'sport' Q847  'tennis' 

queryString = """
SELECT DISTINCT ?obj ?objName 
WHERE { 
  
    wd:Q50932406 ?obj ?r.
        
    
   ?obj sc:name ?objName.
   
    
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/prop/direct/P1424'), ('objName', "topic's main template")]
[('obj', 'http://www.wikidata.org/prop/direct/P2671'), ('objName', 'Google Knowledge Graph ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P279'), ('objName', 'subclass of')]
[('obj', 'http://www.wikidata.org/prop/direct/P641'), ('objName', 'sport')]
[('obj', 'http://www.wikidata.org/prop/direct/P910'), ('objName', "topic's main category")]
5


In [122]:
# wd:Q11459	Serena Williams
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE { 
  
    wd:Q11459 ?p ?obj.
   
   ?p sc:name ?name.
   optional{?obj sc:name ?objName.}
   filter regex(?name, "rank").
   
    
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1352'), ('name', 'ranking'), ('obj', '1')]
1


In [126]:
# P1352  'ranking' 
# wd:Q11459	Serena Williams
queryString = """
SELECT DISTINCT  ?obj ?objName
WHERE { 
  
    wd:Q11459 wdt:P1352 ?obj.
   
    
   optional{?obj sc:name ?objName.}

    
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('obj', '1')]
1


In [130]:
# P1352  'ranking' 
# wd:Q11459	Serena Williams

#P106  'occupation' 
#Q10833314 'tennis player' 
 
queryString = """
SELECT DISTINCT ?player ?playerName ?obj 
WHERE { 
  
    ?player wdt:P1352 ?obj;
        wdt:P106 wd:Q10833314.
    
    ?player sc:name ?playerName.

    
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q138094'), ('playerName', 'Anna Karolína Schmiedlová'), ('obj', '26')]
[('player', 'http://www.wikidata.org/entity/Q15197746'), ('playerName', 'Wesley Koolhof'), ('obj', '5')]
[('player', 'http://www.wikidata.org/entity/Q15197746'), ('playerName', 'Wesley Koolhof'), ('obj', '462')]
[('player', 'http://www.wikidata.org/entity/Q240837'), ('playerName', 'Rika Fujiwara'), ('obj', '13')]
[('player', 'http://www.wikidata.org/entity/Q240837'), ('playerName', 'Rika Fujiwara'), ('obj', '84')]
[('player', 'http://www.wikidata.org/entity/Q4134097'), ('playerName', 'Margarita Gasparyan'), ('obj', '25')]
[('player', 'http://www.wikidata.org/entity/Q7281705'), ('playerName', 'Radu Albot'), ('obj', '56')]
[('player', 'http://www.wikidata.org/entity/Q7281705'), ('playerName', 'Radu Albot'), ('obj', '108')]
[('player', 'http://www.wikidata.org/entity/Q18918709'), ('playerName', 'Markéta Vondroušová'), ('obj', '14')]
[('player', 'http://www.wikidata.or

In [131]:
# P1352  'ranking' 
# wd:Q11459	Serena Williams

#P106  'occupation' 
#Q10833314 'tennis player'  
 
queryString = """
SELECT DISTINCT ?player ?playerName ?obj 
WHERE { 
  
    ?player wdt:P1352 ?obj;
        wdt:P106 wd:Q10833314.
    
    ?player sc:name ?playerName.
    filter (?obj = 1).
    
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q207352'), ('playerName', 'Agnieszka Radwańska'), ('obj', '1')]
[('player', 'http://www.wikidata.org/entity/Q455136'), ('playerName', 'Anders Järryd'), ('obj', '1')]
[('player', 'http://www.wikidata.org/entity/Q455411'), ('playerName', 'Tom Okker'), ('obj', '1')]
[('player', 'http://www.wikidata.org/entity/Q310419'), ('playerName', 'Nenad Zimonjić'), ('obj', '1')]
[('player', 'http://www.wikidata.org/entity/Q53393'), ('playerName', 'Jimmy Connors'), ('obj', '1')]
[('player', 'http://www.wikidata.org/entity/Q10118'), ('playerName', 'Victoria Azarenka'), ('obj', '1')]
[('player', 'http://www.wikidata.org/entity/Q13538'), ('playerName', 'Henri Kontinen'), ('obj', '1')]
[('player', 'http://www.wikidata.org/entity/Q1743002'), ('playerName', 'Robert Seguso'), ('obj', '1')]
[('player', 'http://www.wikidata.org/entity/Q189542'), ('playerName', 'Stefan Edberg'), ('obj', '1')]
[('player', 'http://www.wikidata.org/entity/Q232566'), ('playerName'

Final query for this task

In [133]:
# P1532  'country for sport'
# P1352  'ranking' 
# wd:Q11459	Serena Williams

#P106  'occupation' 
#Q10833314 'tennis player'  
 
queryString = """
SELECT DISTINCT ?country ?countryName  
WHERE { 
  
    ?player wdt:P1352 ?obj;
        wdt:P106 wd:Q10833314;
        wdt:P1532 ?country.
    
    ?country sc:name ?countryName.
    filter (?obj = 1).
    
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q216923'), ('countryName', 'Chinese Taipei')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('countryName', "People's Republic of China")]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('countryName', 'Soviet Union')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('countryName', 'Russia')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('countryName', 'Canada')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('countryName', 'Japan')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q184'), ('countryName', 'Belarus')]
[('country', 'http://www.wikidata.org/entity/Q191'), ('countryName